In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import torch
from datasets import load_dataset, load_metric,concatenate_datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from datasets import Dataset, DatasetDict, load_dataset, load_metric
import torch
import random

In [3]:
def check_requires_grad(model):
    print("Checking which high-level components require gradients:\n")

    # Checking Embeddings
    print("Embeddings:")
    for name, param in model.wrapped_model.bert.embeddings.named_parameters():
        grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
        print(f"  {name} - {grad_status}")

    # Checking Encoder
    print("\nEncoder:")
    for i, layer in enumerate(model.wrapped_model.bert.encoder.layer):
        print(f"  Layer {i}:")
        for name, param in layer.named_parameters():
            grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
            print(f"    {name} - {grad_status}")

    # Checking Attention
    print("\nAttention:")
    for i, layer in enumerate(model.wrapped_model.bert.encoder.layer):
        print(f"  Layer {i} Attention:")
        for name, param in layer.attention.named_parameters():
            grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
            print(f"    {name} - {grad_status}")

    # Checking MLM Head
    print("\nMasked LM Head:")
    for name, param in model.wrapped_model.cls.named_parameters():
        grad_status = "requires gradient" if param.requires_grad else "does NOT require gradient"
        print(f"  {name} - {grad_status}")

    print("\nCheck complete.")


# TEST

In [4]:
import torch
from datasets import load_dataset, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BertConfig, BertModel


In [5]:
import random
torch.cuda.manual_seed(42)
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-medium")
train_data = load_dataset("c4", "en", split='train', streaming=True)
test_data = load_dataset("c4", "en", split='validation', streaming=True)

/media/.venv/lib/python3.11/site-packages/datasets/load.py:1491: FutureWarning: The repository for c4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/c4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/delyanh/.cache/huggingface/modules/datasets_modules/datasets/c4/584d57ebe81c209b6c7f31727066d2c4b4bba37cb7092cdd83083d5ec11207db/c4.py:53: FutureWarning: Dataset 'c4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


In [7]:
from datasets import load_from_disk

load_path = "/home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM"
loaded_dataset_dict = load_from_disk(load_path)

# Access different splits
train_data1 = loaded_dataset_dict["train"]
validation_data1 = loaded_dataset_dict["validation"]

In [8]:

config = BertConfig(
    hidden_size=512,
    num_hidden_layers=8,
    num_attention_heads=8,
    intermediate_size=2048,
    max_position_embeddings=512,
    type_vocab_size=1,  # Adjusted from 2 to 1 based on your config
    vocab_size=30522,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
training_args = TrainingArguments(
    output_dir = "model_BERT",
    evaluation_strategy="steps",
    #save_steps=50,
    eval_steps=3000,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    learning_rate=5e-4,
    weight_decay=0.03,
    logging_steps=5,
    num_train_epochs=1,
    save_strategy  = "no",
    warmup_ratio = 0.15,
    bf16 = True,
    seed = 42
   # max_steps = 4000
    #load_best_model_at_end=True,  # Load the best model at the end
    #metric_for_best_model="eval_f1_macro",   # Metric to compare for best model
    #greater_is_better=True


)

model = AutoModelForMaskedLM.from_config(config).to(torch.bfloat16)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data1,
    eval_dataset=validation_data1,
    tokenizer=tokenizer,
    data_collator = data_collator,




)


trainer.train()
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

/media/.venv/lib/python3.11/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
3000,6.858400,6.752958
6000,6.545700,6.449647
9000,5.149600,5.025860
12000,4.267800,4.156699
15000,4.058400,3.831000
18000,3.789100,3.650591
21000,3.611400,3.527576
24000,3.676600,3.441055
27000,3.472200,3.374135
30000,3.452700,3.327477


Perplexity: 23.90


In [22]:
config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules = target_modules,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model12 = get_peft_model(model12, config)
print(model12.config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 8,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}



In [4]:
from peft_pretraining.relora import ReLoRaModel, ReLoRaLinear, merge_and_reinit_functional
#parameters after grid search 1
from peft import LoraConfig, get_peft_model
import wandb
wandb.init(project="bert-lora-experiment")
config = BertConfig(
    hidden_size=512,
    num_hidden_layers=8,
    num_attention_heads=8,
    intermediate_size=2048,
    max_position_embeddings=512,
    type_vocab_size=1,  # Adjusted from 2 to 1 based on your config
    vocab_size=30522,
)


model = AutoModelForMaskedLM.from_config(config)

target_modules = ["query", "key", "value"]


model = ReLoRaModel(
            model,
            r=64,
            lora_alpha=128,
            lora_dropout=0.1,
            target_modules=["query", "key", "value"],
            trainable_scaling= False,
            keep_original_weights=True,
            lora_only= False,
            quantize=None,
            use_double_quant=False,
        )

wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin


In [2]:
#chocolate-galaxy-701
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 1e-3\
    --max_length 512 \
    --weight_decay 0.02\
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3500 \
    --warmup_steps 9000 \
    --seed 42\
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 15625 \
    --restart_warmup_steps 500 \
    --scheduler cosine_restarts \
    --reset_optimizer_on_relora False \
    --relora 15625\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9\
    --lora_alpha 64\
    --min_lr_ratio 0.000000000000000000000000000000000000000000001\

    



2024-09-21 20:18:57.251 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-21 20:18:57.252 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240921_201858-3brg1tmr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run radiant-breeze-729
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/3brg1tmr
2024-09-21 20:18:59.242 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-21 20:18:59.242 | INFO     | __main__:main:444 - ****************************************


In [ ]:
from peft_pretraining.relora import ReLoRaModel, ReLoRaLinear, merge_and_reinit_functional
#parameters after grid search 1
from peft import LoraConfig, get_peft_model
import wandb
wandb.init(project="bert-lora-experiment")
config = BertConfig(
    hidden_size=512,
    num_hidden_layers=8,
    num_attention_heads=8,
    intermediate_size=2048,
    max_position_embeddings=512,
    type_vocab_size=1,  # Adjusted from 2 to 1 based on your config
    vocab_size=30522,
)


model = AutoModelForMaskedLM.from_config(config)

target_modules = ["query", "key", "value"]


model = ReLoRaModel(
            model,
            r=64,
            lora_alpha=128,
            lora_dropout=0.1,
            target_modules=["query", "key", "value"],
            trainable_scaling= False,
            keep_original_weights=True,
            lora_only= False,
            quantize=None,
            use_double_quant=False,
        )

model.config = config
model.bert = model.wrapped_model.bert  # Directly assigning the bert submodule
model.cls = model.wrapped_model.cls 
print(model)
for name, param in model.named_parameters():
    if any(target_module in name and "base_layer" in name for target_module in target_modules):
        param.requires_grad = False
        #print(f"{name}: gradients disabled (Base layer with LoRA alternative)")
    else:
        param.requires_grad = True
        #print(f"{name}: gradients enabled")
#check_requires_grad(model)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
training_args = TrainingArguments(
    output_dir = "model_BERT",
    evaluation_strategy="steps",
    #save_steps=50,
    eval_steps=3500,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    learning_rate= 5e-4,
    weight_decay=0.02,
    logging_steps=5,
    num_train_epochs=1,
    save_strategy  = "no",
    warmup_ratio = 0.15,
    bf16 = True,
    report_to="wandb",
    seed = 42
    #max_steps = 1000
    #load_best_model_at_end=True,  # Load the best model at the end
    #metric_for_best_model="eval_f1_macro",   # Metric to compare for best model
    #greater_is_better=True


)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data1,
    eval_dataset=validation_data1,
    tokenizer=tokenizer,
    data_collator = data_collator,
    ignore_keys_for_eval=["bert"],



)


trainer.train()
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
wandb.finish()

In [9]:
# earthy-smoke-669
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path bert_c4 \
    --dataset_path /home/delyanh/Projects/ReVeRA/preprocessed_cola_dataset_BERT_MEDIUM \
    --batch_size 64\
    --lr 1e-3\
    --max_length 512 \
    --weight_decay 0.02\
    --num_training_steps 62500 \
    --save_every 600000\
    --eval_every 3500 \
    --warmup_steps 9000 \
    --seed 42\
    --tags relora_bert_medium \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 15625 \
    --restart_warmup_steps 500 \
    --scheduler cosine_restarts \
    --reset_optimizer_on_relora True \
    --relora 15625\
    --lora_r 64\
    --lora_alpha 128\
    


2024-09-21 19:44:34.740 | INFO     | __main__:main:369 - Global rank 0, local rank 0, device: 0
2024-09-21 19:44:34.741 | INFO     | __main__:main:373 - Process group initialized
wandb: Currently logged in as: delyanhristov06 (delyanhristov06-). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /home/delyanh/Projects/ReVeRA/ReLoRa/wandb/run-20240921_194435-k24va5m7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run peachy-breeze-728
wandb: ⭐️ View project at https://wandb.ai/delyanhristov06-/peft_pretraining
wandb: 🚀 View run at https://wandb.ai/delyanhristov06-/peft_pretraining/runs/k24va5m7
2024-09-21 19:44:37.096 | INFO     | __main__:main:443 - Using dist with rank 0 (only rank 0 will log)
2024-09-21 19:44:37.096 | INFO     | __main__:main:444 - ****************************************
2

warm_up_reset 300
warm_up 15000

compare with standart lora to see if everything is working

after that test fewer warm_up steps
different magnitude pruning